In [1]:
import scvelo as scv
import loompy
import igraph as ig
import anndata as ad
import pandas as pd
import re
import scanpy as sc 
import matplotlib.pyplot as plt 
import numpy as np 
import scipy as sp
import seaborn as sb
import harmonypy

#unspliced = pd.read_csv("/omics/groups/OE0540/internal/users/ruehle/rnavelocity/pre_kallisto/hg38_release101/velocity_files_coll/counts/final_unspliced_all_cells.csv", index_col = 0)
#spliced = pd.read_csv("/omics/groups/OE0540/internal/users/ruehle/rnavelocity/pre_kallisto/hg38_release101/velocity_files_coll/counts/final_spliced_all_cells.csv", index_col = 0)

#dummys
unspliced = pd.read_csv("/omics/groups/OE0540/internal/users/ruehle/rnavelocity/pre_kallisto/hg38_release101/velocity_files_coll/counts/subset_unspliced.csv", index_col = 0)
spliced = pd.read_csv("/omics/groups/OE0540/internal/users/ruehle/rnavelocity/pre_kallisto/hg38_release101/velocity_files_coll/counts/subset_spliced.csv", index_col = 0)

In [2]:
spliced = spliced.drop(spliced.columns[0], axis=1) #columns
unspliced = unspliced.drop(unspliced.columns[0], axis =1 ) #columns

spliced = spliced.T
unspliced = unspliced.T

#initaliaze anndata object 
adata = ad.AnnData(spliced, dtype='int32')
adata.layers["X"] = spliced
adata.layers["spliced"] = spliced
adata.layers["unspliced"] = unspliced

In [3]:
adata.obs

""
20287_7#1
20287_7#10
20287_7#11
20287_7#12
20287_7#13
...
25757_6#95
25757_6#96
25757_6#97
25757_6#98


In [4]:
#read in metadata 
cellmeta = pd.read_csv('/icgc/dkfzlsdf/analysis/B260/projects/HipSci/openAccess/endoderm_differentation/metadata/cell_metadata_scendodiff.csv')
methmeta = pd.read_csv('/home/j890e/scvelo/methanno.csv')

cellmeta = pd.concat([cellmeta, methmeta], axis=0)
cellmeta.index = cellmeta['cell'] #name rows after cells 

In [118]:
#---------------------extract cell number from adata.obs of .loom file----------------------

adata.obs['cell'] = adata.obs_names

#-------------------------match cell number with metadata per cell---------------------------------- 

#match with timepoint of collection 
for cellnumber in adata.obs['cell']:
    for cell in cellmeta['cell']: 
        if cellnumber == cell:
            ind = adata.obs.index[adata.obs['cell'] == cellnumber]
            adata.obs.loc[ind, 'timepoint'] =  cellmeta.loc[cellnumber, 'timepoint']
            adata.obs.loc[ind, 'donor'] =  cellmeta.loc[cellnumber, 'donor']
            adata.obs.loc[ind, 'exp_batch'] =  cellmeta.loc[cellnumber, 'batch']
            adata.obs.loc[ind, 'annas_pseudotime'] =  cellmeta.loc[cellnumber, 'pseudotime']

In [16]:
#Extract run 

cellno = []
for filename in adata.obs_names:
    m = re.findall('[0-9]+_', filename)
    cellno.append(m)

flattened_cellno = [y for x in cellno for y in x]
flattened = [sub.replace('_', '') for sub in flattened_cellno]
adata.obs['run'] = flattened_cellno

['20287',
 '20287',
 '20287',
 '20287',
 '20287',
 '20287',
 '20287',
 '20287',
 '20287']

In [118]:
# Filter Nans
bdata = adata[~pd.isna(adata.obs['timepoint']), :]

print("After filtering out cells without metadata: anndata object with" + str(bdata.n_obs) + "lines/cells and " + str(bdata.n_vars) +"columns/genes")

#CPM normalization 
sc.pp.normalize_total(bdata, target_sum=1e6)

#logarithmize
sc.pp.log1p(bdata)

#filter differentially expressed genes 
sc.tl.rank_genes_groups(bdata, groupby='timepoint', use_rep ='X',
                        method='t-test_overestim_var', n_genes=500) # compute differential expression

DE_df = sc.get.rank_genes_groups_df(bdata, group = None)
DE_names = DE_df['names'].tolist()

geneswodoubles = list(dict.fromkeys(DE_names))
bdata = bdata[:, geneswodoubles]

print("Differentially expressed after removing double genes")
print(DE_df)

#calculate some qc metrics for highly variable genes only 
bdata.obs['n_counts_highlyvariable'] = bdata.X.sum(1)
bdata.obs['log_counts_highlyvariable'] = np.log(bdata.obs['n_counts_highlyvariable'])
bdata.obs['n_genes_highlyvariable'] = (bdata.X > 0).sum(1)

#batch correct
#sc.pp.regress_out(bdata, "total_counts")
sc.tl.pca(bdata, n_comps = 30)
sc.external.pp.harmony_integrate(bdata, ['exp_batch', 'run'])
                     

#-------------------------plot pca and umap---------------------------------- 
sc.pp.neighbors(bdata, n_neighbors = 50, n_pcs = 3, use_rep = 'X')                        
#sc.pp.neighbors(bdata, n_neighbors = 50, n_pcs = 30, use_rep = 'X_pca_harmony')
scv.tl.umap(bdata)

sc.pl.embedding(bdata, basis = 'X_pca_harmony', color = "exp_batch", save = "_colorbatch.pdf")
sc.pl.embedding(bdata, basis = 'X_pca', color = 'exp_batch', save = "_colorbatch.pdf")
sc.pl.embedding(bdata, basis = 'X_pca_harmony', color = "timepoint", save = "_timepoint.pdf")
sc.pl.embedding(bdata, basis = 'X_pca', color = 'timepoint', save = "_timepoint.pdf")
sc.pl.embedding(bdata, basis = 'X_pca_harmony', color = "donor", save = "_donor.pdf")
sc.pl.embedding(bdata, basis = 'X_pca', color = 'donor', save = "_donor.pdf")
#sc.pl.embedding(bdata, basis = 'X_pca', color = 'total_counts', save = "_totalcounts.pdf")
#sc.pl.embedding(bdata, basis = 'X_pca', color = 'n_counts_highlyvariable', save = "_ncounts_highlyvariable.pdf")
#sc.pl.embedding(bdata, basis = 'X_pca', color = 'n_genes_highlyvariable', save = "_ngenes_highlyvariable.pdf")

sc.pl.embedding(bdata, basis = 'umap', color = 'timepoint', save = "_correctedpca3_timepoint.pdf")
sc.pl.embedding(bdata, basis = 'umap', color = 'exp_batch', save = "_correctedpca3_batch.pdf")

sc.tl.louvain(bdata, key_added = "louvain_1.0") # default resolution in 1.0
sc.tl.louvain(bdata, resolution = 0.6, key_added = "louvain_0.6")
sc.tl.louvain(bdata, resolution = 0.4, key_added = "louvain_0.4")
sc.tl.louvain(bdata, resolution = 1.4, key_added = "louvain_1.4")

sc.pl.umap(bdata, color=['louvain_0.4', 'louvain_0.6', 'louvain_1.0','louvain_1.4'], save = "louvain.pdf")
sc.pl.embedding(bdata, basis = 'X_pca_harmony', color = ['louvain_0.4', 'louvain_0.6', 'louvain_1.0','louvain_1.4'], save = "louvain_corrected.pdf" )

#Plot louvain clusters and QC metrics
#sc.pl.violin(bdata, 'mt_frac', groupby= 'louvain_1.0', save = "mt_frac_louvain1.0.pdf")
#sc.pl.violin(bdata, "total_counts", size=2, log=True, cut=0, groupby= 'louvain_1.0', save = "total_counts_louvain1.0.pdf")
#sc.pl.violin(bdata, 'n_genes_by_counts', size=2, log=True, cut=0, groupby= 'louvain_1.0', save = "n_genes_louvain1.0.pdf")


#-------------------------velocity plots---------------------------------- 

scv.pp.moments(bdata, n_neighbors = 50, n_pcs = 3, use_rep = "X") #mu and ms first and second order moments 
scv.tl.recover_dynamics(bdata)
scv.tl.velocity(bdata, mode='dynamical')
scv.tl.velocity_graph(bdata) #based on distances 

scv.pl.velocity_embedding(bdata, basis='umap', color = 'timepoint', legend_loc = 'right margin', save = "velocity1_correctedumap_timepoint.pdf")
scv.pl.velocity_embedding_grid(bdata, basis='umap', color = 'timepoint', legend_loc = 'right margin',save = "velocity2_correctedumap_timepoint.pdf" )
scv.pl.velocity_embedding_stream(bdata, basis='umap', color = 'timepoint', cutoff_perc = 0, legend_loc = 'right margin', min_mass = 2, save = "velocity3_correctedumap_timepoint_min_mass2.png")

scv.pl.velocity_embedding(bdata, basis='X_pca_harmony', color = 'timepoint', legend_loc = 'right margin', save = "velocity1_correctedpca_timepoint.pdf")
scv.pl.velocity_embedding_grid(bdata, basis='X_pca_harmony', color = 'timepoint', legend_loc = 'right margin', save = "velocity2_correctedpca_timepoint.pdf")
scv.pl.velocity_embedding_stream(bdata, basis='X_pca_harmony', color = 'timepoint', cutoff_perc = 0, legend_loc = 'right margin', min_mass = 2, save = "velocity3_correctedpca_timepoint_min_mass2.png")

scv.pl.velocity_embedding(bdata, basis='X_pca', color = 'timepoint', legend_loc = 'right margin', save = "velocity1_correctedpca_timepoint.pdf")
scv.pl.velocity_embedding_grid(bdata, basis='X_pca', color = 'timepoint', legend_loc = 'right margin', save = "velocity2_correctedpca_timepoint.pdf")
scv.pl.velocity_embedding_stream(bdata, basis='X_pca', color = 'timepoint', cutoff_perc = 0, legend_loc = 'right margin', min_mass = 3, save = "velocity3_correctedpca_timepoint_min_mass2.png")

#-------------------------further velocity analysis---------------------------------- 

#identify important genes
scv.tl.rank_velocity_genes(bdata, groupby = 'timepoint', min_corr=.3)
df = scv.DataFrame(bdata.uns['rank_velocity_genes']['names'])
#print("The 50 most important genes identified per cluster (timepoint) ")
#print(df.head(50))

#scv.pl.velocity(bdata, ['GATA6', 'NANOG', 'T'], basis = 'X_pca_harmony', color = 'timepoint', save = "GATA6NANOGT_correctedpca_timepoint.pdf")

#top_genes = bdata.var['fit_likelihood'].sort_values(ascending=False).index
#scv.pl.scatter(bdata, basis=top_genes[:10], ncols=5, frameon=False, color='timepoint', save = "scatter_top10velospertimepoint_corrected_timepoint.pdf")

#-------------------------pseudotime vs latent time---------------------------------- 
scv.tl.velocity_pseudotime(bdata) #calculated based on velocity graph 
scv.tl.latent_time(bdata)

scv.pl.scatter(bdata, basis = 'X_pca_harmony', color='latent_time', color_map='gnuplot', size=80, save = "latenttime_pca_corrected.pdf")
scv.pl.scatter(bdata, basis = 'X_pca_harmony', color='velocity_pseudotime', color_map='gnuplot', save = "pseudotime_pca_corrected.pdf")

scv.pl.scatter(bdata, basis = 'umap', color='latent_time', color_map='gnuplot', save = "latenttime_umap_corrected.pdf")
scv.pl.scatter(bdata, basis = 'umap', color='velocity_pseudotime', color_map='gnuplot', save = "pseudotime_umap_corrected.pdf")

scv.pl.scatter(bdata, basis = 'umap', color='annas_pseudotime', color_map='gnuplot', save = "annaspseudotime_umap_corrected.pdf")
scv.pl.scatter(bdata, basis = 'X_pca_harmony', color='annas_pseudotime', color_map ='gnuplot', save = "annaspseudotime_pca_corrected.pdf" )

#top velocity genes (overall genes ) sorted by latent_time
#top_genes = bdata.var['fit_likelihood'].sort_values(ascending=False).index[:300]
#scv.pl.heatmap(bdata, var_names=top_genes, sortby='velocity_pseudotime', col_color='timepoint', n_convolve=100, save = "heatmap_top100velos_corrected.pdf")

#save anndata object 
bdata.write(filename="/omics/groups/OE0540/internal/users/ruehle/rnavelocity/pre_kallisto/hg38_release101/velocity_files_coll/scvelo_output/use/kallisto_cpmtop2000DE_neighbors_repX.h5ad")

KeyboardInterrupt: 

64677


AnnData object with n_obs × n_vars = 64677 × 9
    obs: 'cell'
    layers: 'X', 'spliced', 'unspliced'

['0', '2', '8', '7', '_', '7', '#', '1', '2']

KeyError: ('20287_7#1', 'timepoint')

In [111]:
cellmeta.head()

,cell,batch,pseudotime,timepoint,donor
0,21843_1#10,expt_09,0.302240,day1,joxm_1
1,21843_1#100,expt_09,0.504279,day1,fafq_1
2,21843_1#101,expt_09,0.435249,day1,fafq_1
3,21843_1#102,expt_09,0.269503,day1,wuye_2
4,21843_1#103,expt_09,0.373565,day1,joxm_1


In [116]:
cellmeta.loc["21843_1#103",:]

KeyError: '21843_1#103'